In [30]:
import sqlite3
import pandas as pd
import csv
import argparse

In [31]:
db = sqlite3.connect("Covid.db")
cur = db.cursor()
con = sqlite3.connect("Covid.db")


In [32]:
df = pd.read_csv(r"/Users/jenny.wade@schibsted.com/Documents/GitHub/python_programmering/Inlämning/covid_vaccinations.csv")
df = df.fillna(0)
db = df.to_sql("Covid_vaccinations", con, schema=None, if_exists="replace", index=True, index_label=None, chunksize=None, dtype=None, method=None)


In [21]:
#Skapa dataframes från queries i en linje kod
df = pd.read_sql_query("SELECT * FROM Covid_vaccinations", con)
print(df.head())

   index  Unnamed: 0      country iso_code        date  total_vaccinations  \
0      0           0  Afghanistan      AFG  2021-02-22                 0.0   
1      1           1  Afghanistan      AFG  2021-02-23                 0.0   
2      2           2  Afghanistan      AFG  2021-02-24                 0.0   
3      3           3  Afghanistan      AFG  2021-02-25                 0.0   
4      4           4  Afghanistan      AFG  2021-02-26                 0.0   

   people_vaccinated  people_fully_vaccinated  daily_vaccinations  \
0                0.0                      0.0                 0.0   
1                0.0                      0.0              1367.0   
2                0.0                      0.0              1367.0   
3                0.0                      0.0              1367.0   
4                0.0                      0.0              1367.0   

   total_vaccinations_per_hundred  ...  Covaxin  CanSino  Abdala  \
0                             0.0  ...        0 

In [ ]:
#Creating all tables (empty) I need for my database below:

In [33]:
cur.execute("""CREATE TABLE Daily_Vaccine_Data(
            iso_date text PRIMARY KEY,
            country text,
            daily_vaccinations integer,
            daily_vaccinations_per_million)""")
con.commit()

In [34]:
cur.execute("""CREATE TABLE Country_Source(
            country text PRIMARY KEY, 
            source_name text, 
            source_website text)""")
con.commit()

In [35]:
cur.execute("""CREATE TABLE Country_Vaccines(
    country text PRIMARY KEY, Johnson_and_Johnson integer, 
    Oxford_AstraZeneca integer, Sinovac integer,
    Sputnik_V integer, Moderna integer, 
    Sinopharm_Beijing integer, Covaxin integer, 
    CanSino integer, Abdala integer, 
    Pfizer_BioNTech integer, QazVac integer, 
    EpiVacCorona integer, Soberana02 integer, 
    Sinopharm_HayatVax integer, RBDDimer integer, 
    Sinopharm_Wuhan integer)""")
con.commit()

In [36]:
cur.execute("""CREATE TABLE Country_Total(
    total_vaccinations integer PRIMARY KEY, 
    country text , 
    people_vaccinated integer, 
    people_fully_vaccinated integer,
    total_vaccinations_per_hundred real, 
    people_vaccinated_per_hundred real, 
    people_fully_vaccinated_per_hundred)""")
con.commit()

In [ ]:
#Creating copy tables with all the data I need for each table already ordered

In [39]:
cur.execute("""CREATE TABLE Daily_Vaccine_Data_copy AS SELECT
            iso_date, country, daily_vaccinations, daily_vaccinations_per_million
            FROM Covid_vaccinations""")
con.commit()

In [40]:
cur.execute("""CREATE TABLE Country_Source_data
            AS SELECT DISTINCT country, source_name, source_website 
            FROM Covid_vaccinations""")
con.commit()

In [41]:
cur.execute("""CREATE TABLE Country_vaccines_data AS SELECT DISTINCT country, 
            Johnson_and_Johnson, Oxford_AstraZeneca, Sinovac, Sputnik_V, Moderna, 
            Sinopharm_Beijing, Covaxin, CanSino, Abdala, Pfizer_BioNTech, QazVac, 
            EpiVacCorona, Soberana02, Sinopharm_HayatVax, RBD_Dimer, Sinopharm_Wuhan 
            FROM Covid_vaccinations""")
con.commit()

In [46]:
cur.execute("DROP TABLE Country_Total_data")
con.commit()

In [44]:
cur.execute("""CREATE TABLE Country_Total_data AS SELECT DISTINCT max(total_vaccinations), country, people_vaccinated
            people_fully_vaccinates, total_vaccinations_per_hundred, people_vaccinated_per_hundred,
            people_fully_vaccinated_per_hundred FROM Covid_vaccinations""")
con.commit()

In [45]:
#Below is the code I'm trying to use in order to fill my tables, but it's not working
cur.execute("""INSERT INTO Country_Vaccines SELECT * FROM Country_Vaccines_data""")
con.commit()